## Setup
Importing the necessary modules and functions. 

In [1]:
from SPARQLWrapper import SPARQLWrapper, JSON
from pyvis.network import Network
from IPython.display import display, HTML, IFrame
import urllib
import simplejson as json
import pandas as pd
import re
import requests
import warnings
import statistics
from edelweiss_data import API, QueryExpression as Q

pd.set_option("display.max_colwidth", -1)
warnings.filterwarnings("ignore", category = UserWarning)

/tmp/ipykernel_131591/150251967.py:13: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


## Workflow: Extracting and Analyzing Data Related to an AOP of Interest

The goal of this workflow is to find biological/chemical information related to an Adverse Outcome Pathway (AOP). The workflow starts by specifying the AOP (the pathway with the ID number 37 in this example). Then, by using several services, we will collect data relevant to this pathway. The workflow then ends with a pathway analysis using the transcriptomics data from TG-GATES and the molecular pathways in WikiPathways to identify the pathways that are significantly affected with the exposure to the chemicals of interest. 

## Defining the AOP of Interest
Here we define the specific AOP that we want to analyze in this workflow based on its ID number, 37. More info to this pathway can be seen from the AOP-Wiki website: https://aopwiki.org/aops/37.

In [2]:
AOPid = "37"

The set of the AOP identifiers can be seen here: https://aopwiki.org/aops?direction=asc&sort=id

## Setting the service URLs
As aforementioned, we need to utilize several online services throughout this workflow. Here, we set the URLs to these services as Python objects for an easiness of use. Note that though, some of the services may have been removed (such as the ones hosted by the OpenRiskNet). finding another service that can accomplish the same goal is needed, get their source data/docker images and manually deploy them, or some other way to complete that step.

In [5]:
# SPARQL endpoint URLs
aopwikisparql = SPARQLWrapper("http://aopwiki.cloud.vhp4safety.nl/sparql/")
aopdbsparql = SPARQLWrapper("http://aopdb.rdf.bigcat-bioinformatics.org/sparql/") 
wikipathwayssparql = SPARQLWrapper("http://sparql.wikipathways.org/sparql/")

# ChemIdConvert URL
chemidconvert = "https://chemidconvert.cloud.douglasconnect.com/v1/"

# BridgeDB base URL
bridgedb = "http://bridgedb.cloud.vhp4safety.nl/"

# EdelweissData API URL
edelweiss_api_url = "https://api.staging.kit.cloud.douglasconnect.com"

## AOP-Wiki RDF 
The AOP-Wiki is a repository where AOP-related information are stored along with key events, key event relationships, and stressors that allow the reuse of toxicological knowledge for risk assessors. AOP-Wiki RDF then converts the XML files reported by the AOP-Wiki into RDF formatted files which has been exposed in a public SPARQL endpoint.

In [7]:
# Defining all variables as ontology terms present in AOP-Wiki RDF.
title = 'dc:title'
webpage = 'foaf:page'
creator = 'dc:creator'
abstract = 'dcterms:abstract'
key_event = 'aopo:has_key_event'
molecular_initiating_event = 'aopo:has_molecular_initiating_event'
adverse_outcome = 'aopo:has_adverse_outcome'
key_event_relationship = 'aopo:has_key_event_relationship'
stressor = 'ncit:C54571'

# Storing the terms as a list object. 
listofterms = [title,webpage,creator,abstract,key_event,molecular_initiating_event,adverse_outcome,key_event_relationship,stressor]

# Initiating the DataFrame.
AOPinfo = pd.DataFrame(columns=['Properties'], index = [list(listofterms)])

# Running the whole query. 
for term in listofterms:
    sparqlquery = '''
    PREFIX ncit: <http://ncicb.nci.nih.gov/xml/owl/EVS/Thesaurus.owl#>
    SELECT (group_concat(distinct ?item;separator=";") as ?items)
    WHERE{
    ?AOP_URI a aopo:AdverseOutcomePathway;'''+term+''' ?item.
    FILTER (?AOP_URI = aop:'''+AOPid +''')}
    '''
    aopwikisparql.setQuery(sparqlquery)
    aopwikisparql.setReturnFormat(JSON)  
    results = aopwikisparql.query().convert()
    for result in results["results"]["bindings"]:
        if 'identifiers.org' in result["items"]["value"]:
            AOPinfo.at[term,'Properties'] = ', '.join(result["items"]["value"].split(';'))
        else:
            AOPinfo.at[term,'Properties'] = result["items"]["value"]

display(AOPinfo)

,Properties
dc:title,PPARα activation leading to hepatocellular adenomas and carcinomas in rodents
foaf:page,https://identifiers.org/aop/37
dc:creator,"J. Christopher Corton, Cancer AOP Workgroup. National Health and Environmental Effects Research Laboratory, Office of Research and Development, Integrated Systems Toxicology Division, US Environmental Protection Agency, Research Triangle Park, NC. Corresponding author for wiki entry (corton.chris@epa.gov)\n"
dcterms:abstract,"Several therapeutic agents and industrial chemicals induce liver tumors in rats and mice through the activation of the peroxisome proliferator-activated receptor alpha (PPAR&alpha;). The molecular and cellular events by which PPAR&alpha; activators induce rodent hepatocarcinogenesis have been extensively studied and elucidated. The weight of evidence relevant to the hypothesized AOP for PPAR&alpha; activator-induced rodent hepatocarcinogenesis is summarized here. Chemical-specific and mechanistic data support concordance of temporal and dose&ndash;response relationships for the key events associated with many PPAR&alpha; activators including a phthalate ester plasticizer di(2-ethylhexyl)phthalate (DEHP) and the drug gemfibrozil. The key events (KE) identified include the MIE &ndash; PPAR&alpha; activation measured as a characteristic change in gene expression,&nbsp;&nbsp;KE2&nbsp;&ndash; increased enzyme activation, characteristically those involved in lipid metabolism and cell cycle control, KE3&nbsp;&ndash; increased cell proliferation, KE4 &ndash; selective clonal expansion of preneoplastic foci, and the AO &ndash; &nbsp;&ndash; increases in hepatocellular adenomas and carcinomas. &nbsp;Other biological&nbsp;factors modulate the effects of PPAR&alpha; activators.These modulating events include increases in oxidative stress, activation of NF-kB, and inhibition of gap junction intercellular communication. The occurrence of hepatocellular adenomas and carcinomas is specific to mice and rats. The occurrence of the various KEs in&nbsp;hamsters, guinea pigs,&nbsp;cynomolgous monkeys are generally absent.\n"
aopo:has_key_event,"https://identifiers.org/aop.events/1170, https://identifiers.org/aop.events/1171, https://identifiers.org/aop.events/227, https://identifiers.org/aop.events/716, https://identifiers.org/aop.events/719"
aopo:has_molecular_initiating_event,https://identifiers.org/aop.events/227
aopo:has_adverse_outcome,https://identifiers.org/aop.events/719
aopo:has_key_event_relationship,"https://identifiers.org/aop.relationships/1229, https://identifiers.org/aop.relationships/1230, https://identifiers.org/aop.relationships/1232, https://identifiers.org/aop.relationships/1239, https://identifiers.org/aop.relationships/2252, https://identifiers.org/aop.relationships/2253, https://identifiers.org/aop.relationships/2254"
ncit:C54571,"https://identifiers.org/aop.stressor/11, https://identifiers.org/aop.stressor/175, https://identifiers.org/aop.stressor/191, https://identifiers.org/aop.stressor/205, https://identifiers.org/aop.stressor/206, https://identifiers.org/aop.stressor/207, https://identifiers.org/aop.stressor/208, https://identifiers.org/aop.stressor/210, https://identifiers.org/aop.stressor/211"


## Generating AOP Network

In [8]:
# Generate network from AOP + interlinked AOPs.
Key_Events = str(AOPinfo.iat[4,0]).split(', ')

# A network of pathways can be built by searching other pathways where the key 
# events in our AOP exist. Here, we are searching for other AOPs that include 
# the KEs in our pathway.
for Key_Event in Key_Events:
    sparqlquery = '''
    SELECT ?MIE_ID ?KE_ID ?AO_ID ?KER_ID ?KE_Title
    WHERE{
    ?KE_URI a aopo:KeyEvent; dcterms:isPartOf ?AOP_URI.
    ?AOP_URI aopo:has_key_event ?KE_URI2; aopo:has_molecular_initiating_event ?MIE_URI; aopo:has_adverse_outcome ?AO_URI; aopo:has_key_event_relationship ?KER_URI.
    ?KE_URI2 rdfs:label ?KE_ID; dc:title ?KE_Title. 
    ?MIE_URI rdfs:label ?MIE_ID.
    ?AO_URI rdfs:label ?AO_ID.
    ?KER_URI rdfs:label ?KER_ID.    
    FILTER (?KE_URI = <'''+Key_Event+'''>)}
    '''
    aopwikisparql.setQuery(sparqlquery)
    aopwikisparql.setReturnFormat(JSON)  
    results = aopwikisparql.query().convert()

    MIEs = set([])
    KEs = set([])
    KEtitle = {}
    AOs = set([])
    KERs = set([])
    for result in results["results"]["bindings"]:
        MIEs.add(result["MIE_ID"]["value"])
        AOs.add(result["AO_ID"]["value"])
        KEs.add(result["KE_ID"]["value"])
        KERs.add(result["KER_ID"]["value"])
        KEtitle[result["KE_ID"]["value"]]=result["KE_Title"]["value"]

# Listing all intermediate KEs that are not MIEs or AOs. 
KEsIntermediate = []
for item in KEs:
    if item not in MIEs and item not in AOs:
        KEsIntermediate.append(item) 

# Initiating the network figure
net = Network(height="100%", width="100%")

# Adding nodes for Molecular Initiating Events, Key Events, and Adverse 
# Outcomes to the network figure
for MIE in MIEs:
    net.add_node(MIE, color = 'lightgreen', size = 50, shape = 'circle', font = '20px arial black', title = KEtitle[MIE])
for KE in KEsIntermediate:
    net.add_node(KE, color = 'khaki', size = 50, shape = 'circle', font = '20px arial black', title = KEtitle[KE])
for AO in AOs:
    net.add_node(AO, color = 'salmon', size = 50, shape = 'circle', font = '20px arial black', title = KEtitle[AO])

# Adding all Key Event Relationships to the network figure after querying all 
# KERs for all KEs in AOP-Wiki RDF
for KER in KERs:
    sparqlquery = '''
    SELECT ?KE_UP_ID ?KE_DOWN_ID 
    WHERE{
    ?KER_URI a aopo:KeyEventRelationship; rdfs:label ?KER_ID; aopo:has_upstream_key_event ?KE_UP_URI; aopo:has_downstream_key_event ?KE_DOWN_URI.
    ?KE_UP_URI rdfs:label ?KE_UP_ID.
    ?KE_DOWN_URI rdfs:label ?KE_DOWN_ID.
    FILTER (?KER_ID = "'''+KER+'''")}
    '''
    aopwikisparql.setQuery(sparqlquery)
    aopwikisparql.setReturnFormat(JSON)  
    results = aopwikisparql.query().convert()
    for result in results["results"]["bindings"]:
        net.add_edge(result["KE_UP_ID"]["value"],result["KE_DOWN_ID"]["value"],width = 2, color = 'black',label = KER, arrows = 'to')

net.show('mygraph.html')
IFrame(src='./mygraph.html', width=700, height=600)

In [10]:
# Querying all chemicals that are part of the selected AOP
sparqlquery = '''
SELECT ?CAS_ID (fn:substring(?CompTox,33) as ?CompTox_ID) ?Chemical_name
WHERE{
?AOP_URI a aopo:AdverseOutcomePathway; nci:C54571 ?Stressor.
?Stressor aopo:has_chemical_entity ?Chemical.
?Chemical cheminf:000446 ?CAS_ID; dc:title ?Chemical_name.
OPTIONAL {?Chemical cheminf:000568 ?CompTox.}
FILTER (?AOP_URI = aop:'''+AOPid +''')}
'''
aopwikisparql.setQuery(sparqlquery)
aopwikisparql.setReturnFormat(JSON)  
results = aopwikisparql.query().convert()

Chemical_names = {}
CompTox = {}

for result in results["results"]["bindings"]:
    try: CompTox[result["CAS_ID"]["value"]] =result["CompTox_ID"]["value"]
    except: pass
for result in results["results"]["bindings"]:
    try: Chemical_names[result["CAS_ID"]["value"]] =result["Chemical_name"]["value"]
    except: pass

Chemdata = pd.DataFrame(columns=['Chemical_name','CAS_ID','CompTox_ID'])
for CAS_ID in Chemical_names:
    Chemdata = Chemdata.append({
        'Chemical_name' : Chemical_names[CAS_ID],
        'CAS_ID'        : CAS_ID,
        'CompTox_ID'    : CompTox[CAS_ID],
        }, ignore_index=True)
display(Chemdata)   # These are all the chemicals included

/tmp/ipykernel_131591/3443788425.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Chemdata = Chemdata.append({
/tmp/ipykernel_131591/3443788425.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Chemdata = Chemdata.append({
/tmp/ipykernel_131591/3443788425.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Chemdata = Chemdata.append({
/tmp/ipykernel_131591/3443788425.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Chemdata = Chemdata.append({
/tmp/ipykernel_131591/3443788425.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  Chemdata 

,Chemical_name,CAS_ID,CompTox_ID
0,Di(2-ethylhexyl) phthalate,117-81-7,DTXSID5020607
1,Gemfibrozil,25812-30-0,DTXSID0020652
2,Nafenopin,3771-19-5,DTXSID8020911
3,Bezafibrate,41859-67-0,DTXSID3029869
4,Fenofibrate,49562-28-9,DTXSID2029874
5,Pirinixic acid,50892-23-4,DTXSID4020290
6,Ciprofibrate,52214-84-3,DTXSID8020331
7,Clofibrate,637-07-0,DTXSID3020336


In [11]:
compounds = []
for index, row in Chemdata.iterrows():
    compounds.append(row['CAS_ID'])
compounds

['117-81-7',
 '25812-30-0',
 '3771-19-5',
 '41859-67-0',
 '49562-28-9',
 '50892-23-4',
 '52214-84-3',
 '637-07-0']

## ChemIdConvert
The ChemIdConvert is a tool to convert chemical descriptors between various formats such as SMILES and InChl through a REST API. 

In [12]:
compoundstable = pd.DataFrame(columns=['CAS_ID', 'Image', 'Smiles', 'InChIKey'])

# Fill "compounds" with the "smiles" by the compound name.
for compound in compounds:
    smiles = requests.get(chemidconvert + 'cas/to/smiles', params={'cas': compound}).json()['smiles']
    inchikey = requests.get(chemidconvert + 'smiles/to/inchikey', params={'smiles': smiles}).json()['inchikey']
    compoundstable = compoundstable.append({'CAS_ID': compound, 'Image': smiles, 'Smiles': smiles, 'InChIKey' : inchikey}, ignore_index=True)

def smiles_to_image_html(smiles):  # "smiles" shadows "smiles" from outer scope, use this function only in "to_html().
    """Gets for each smile the image, in HTML.
    :param smiles: Takes the “smiles” form “compounds”.
    :return: The HTML code for the image of the given smiles.
    """
    return '<img style="width:150px" src="' + chemidconvert+'asSvg?smiles='+urllib.parse.quote(smiles)+'"/>'


# Return a HTML table of "compounds", after "compounds" is fill by "smiles_to_image_html".
HTML(compoundstable.to_html(escape=False, formatters=dict(Image=smiles_to_image_html)))

/tmp/ipykernel_131591/2175663238.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compoundstable = compoundstable.append({'CAS_ID': compound, 'Image': smiles, 'Smiles': smiles, 'InChIKey' : inchikey}, ignore_index=True)
/tmp/ipykernel_131591/2175663238.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compoundstable = compoundstable.append({'CAS_ID': compound, 'Image': smiles, 'Smiles': smiles, 'InChIKey' : inchikey}, ignore_index=True)
/tmp/ipykernel_131591/2175663238.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compoundstable = compoundstable.append({'CAS_ID': compound, 'Image': smiles, 'Smiles': smiles, 'InChIKey' : inchikey}, ignore_index=True)
/tmp/ipykernel_131591/2175663238.py:7: FutureWarning: The frame.

,CAS_ID,Image,Smiles,InChIKey
0,117-81-7,,CCCCC(CC)COC(=O)c1ccccc1C(=O)OCC(CC)CCCC,BJQHLKABXJIVAM-UHFFFAOYSA-N
1,25812-30-0,,Cc1ccc(C)c(OCCCC(C)(C)C(O)=O)c1,HEMJJKBWTPKOJG-UHFFFAOYSA-N
2,3771-19-5,,CC(C)(Oc1ccc(cc1)C2CCCc3ccccc23)C(O)=O,XJGBDJOMWKAZJS-UHFFFAOYSA-N
3,41859-67-0,,CC(C)(Oc1ccc(CCNC(=O)c2ccc(Cl)cc2)cc1)C(O)=O,IIBYAHWJQTYFKB-UHFFFAOYSA-N
4,49562-28-9,,CC(C)OC(=O)C(C)(C)Oc1ccc(cc1)C(=O)c2ccc(Cl)cc2,YMTINGFKWWXKFG-UHFFFAOYSA-N
5,50892-23-4,,Cc1cccc(Nc2cc(Cl)nc(SCC(O)=O)n2)c1C,SZRPDCCEHVWOJX-UHFFFAOYSA-N
6,52214-84-3,,CC(C)(Oc1ccc(cc1)C2CC2(Cl)Cl)C(O)=O,KPSRODZRAIWAKH-UHFFFAOYSA-N
7,637-07-0,,CCOC(=O)C(C)(C)Oc1ccc(Cl)cc1,KNHUKKLJHYUCFP-UHFFFAOYSA-N


## AOP-DB RDF

AOP-DB assists the researchers to discover and develop putative and potential AOPs by integrating AOPs with gene targets, chemicals, diseases, tissues, pathways, species orthology information, ontologies, and gene interactions based on public annotations. AOP-DB RDF, in turn, converts the results into an RDF schema and expose its data in a Virtuoso SPARQL endpoint. All genes related to the AOP of interest are extracted through this process.

In [13]:
Key_Events = str(AOPinfo.iat[4,0]).split(', ')
Genes = []

# Getting AOPs from the KEs.
for Key_Event in Key_Events:
    sparqlquery = '''
    SELECT DISTINCT ?KE_ID ?Entrez_ID WHERE{
    ?KE_URI edam:data_1027 ?Entrez_URI. ?Entrez_URI edam:data_1027 ?Entrez_ID.
    FILTER (?KE_URI = <'''+Key_Event +'''>)}
    '''
    aopdbsparql.setQuery(sparqlquery)
    aopdbsparql.setReturnFormat(JSON)  
    results = aopdbsparql.query().convert()
    for result in results["results"]["bindings"]:
        Genes.append(result["Entrez_ID"]["value"])

print(Genes)

['5465', '403654', '19013', '25747']


## BridgeDb to map identifiers

BridgeDb assists the researchers to explore various biological databases to detect different biological identifiers associated with an identifier at hand.  

In [14]:
inputdatasource = 'L'
outputdatasource = ['H','En']
Species = ['Human','Mouse','Rat']

Mappings = {}
HGNC = []

for source in outputdatasource:
    Mappings[source] = []
    for Entrez in Genes:
        for species in Species:
            allmappings = re.split('\t|\n', requests.get(bridgedb + species + '/xrefs/' + inputdatasource + '/' + Entrez + '?dataSource='+source).text)
            if allmappings[0] != '':
                break
        Mappings[source].append(allmappings[0])

ids = {}
for source in Mappings:
    ids[source]=[]
    for identifier in Mappings[source]:
        ids[source].append(identifier)

GenesTable = pd.DataFrame(columns=['Entrez','HGNC','Ensembl'])
GenesTable['Entrez'] = Genes
GenesTable['HGNC'] = ids['H']
GenesTable['Ensembl'] = ids['En']

display(GenesTable)

,Entrez,HGNC,Ensembl
0,5465,PPARA,ENSG00000186951
1,403654,,
2,19013,,ENSMUSG00000022383
3,25747,,ENSRNOG00000021463
